In [ ]:
MODEL_ID = "unsloth/gemma-2-2b-it"
TOKENIZER_ID = "unsloth/gemma-2-2b-it"

In [ ]:
!pip install -q transformers==4.47.1 
!pip install -q torch==2.5.1+cu121 --index-url https://download.pytorch.org/whl/cu121
!pip install -q triton
!pip install -q scikit-learn
!pip install -q vllm > /dev/null 2>&1
!pip uninstall -y -q pynvml
!pip install -q nvidia-ml-py
!pip install -U -q ipywidgets
!pip install -U -q bitsandbytes
!pip install -U -q "optimum>=1.20.0"
!pip install -q auto_gptq

In [ ]:
from dotenv import load_dotenv

load_dotenv()

In [ ]:
zero_shot_prompt = """
Ти лінгвістичний експерт.
Твоє завдання - знайти усі унікальні іменовані сутності з тексту та повернути їх.

Типи сутностей:
- ART: Артефакт – це об'єкт, створений людиною, наприклад, інструменти, мистецькі твори чи культурні предмети
- DATE: Дата – це конкретна календарна дата.
- DOC: Документ – це офіційний чи неофіційний текст, наприклад, контракт, звіт чи лист.
- JOB: Посада – це професійний титул, назва професії чи назва роботи.
- LOC: Локація – це будь-які географічні назв.
- MON: Гроші – це кількість грошей включно з валютою.
- ORG: Організація – це назва та абревіатура будь-якої компанії.
- PCT: Відсоток – це числове значення у відсотках.
- PERIOD: Період – це проміжок часу чи тривалість.
- PERS: Ім'я – це ім'я будь-якої персони.
- QUANT: Кількість – це вимірювана величина: вага, відстань чи розмір.
- TIME: Час – це конкретний час доби.
- MISC: Різне – сутності, які не підпадають під інші категорії.

Вхідний текст:
'{text}'
"""

In [ ]:
few_shot_prompt = """
You are a Named Entity Recognition (NER) model specialized in Ukrainian language processing.

Your task is to extract all named entities from the given text and classify them strictly into one of the predefined categories below. If an entity does not belong to any of the categories, exclude it from the output.

### Entity Categories:
ORG (proper names of organizations, e.g., NASA, ОДА)  
PERS (person names, e.g., Гаррі Поттер, Дракула)  
LOC (e.g., Україна, г. Еверест)  
MON (e.g., $40, 1 мільйон гривень)  
PCT (e.g., 10%, 99%)  
DATE (e.g., минулого тижня, 10.12.1999)  
TIME (e.g., 18:30)  
PERIOD (e.g., 2014-2015, декілька місяців)  
JOB (professions, e.g., консультант, лікар)

### Examples:

Example 1:  
Text: 'Антон Іванович Муха працює продавцем 26 вересня о 18:00 на Чорну Пʼятницю у компанії Київстар у Києві з кредитом у 1000 грн . під 10 % на 1 р . у банку.'  
Output:  
PERS: 'Антон Іванович Муха'  
JOB: 'продавцем'  
DATE: '26 вересня'  
TIME: '18:00'  
ORG: 'Київстар'  
LOC: 'Києві'  
MON: '1000 грн .'  
PCT: '10 %'  
PERIOD: '1 р .'

Example 2:  
Text: 'Валентина Терешкова стала першою жінкою-космонавтом у 1963 році. Її політ тривав 3 доби, що стало історичною подією для НАСА.'  
Output:  
PERS: 'Валентина Терешкова'  
JOB: 'жінкою-космонавтом'  
DATE: '1963 році'  
PERIOD: '3 доби'  
ORG: 'NASA'

Example 3:  
Text: 'У 2024 році ціни на бензин компанії БензинЗаправка зросли до 50 грн за літр, що склало збільшення на 20 % у порівнянні з минулим роком.'  
Output:  
DATE: '2024 році'  
ORG: 'БензинЗаправка'  
MON: '50 грн'  
PCT: '20 %'

Example 4:  
Text: 'Рекордний дохід у компанії SpaceX у 2023 році склав $ 5 мільярдів.'  
Output:  
ORG: 'SpaceX'  
DATE: '2023 році'  
MON: '$ 5 мільярдів'

Example 5:  
Text: 'Подія відбудеться 12 грудня о 16:30 у Львові в офісі компанії ПриватБанк.'  
Output:  
DATE: '12 грудня'  
TIME: '16:30'  
LOC: 'Львові'  
ORG: 'ПриватБанк'

Example 6:  
Text: 'Микола Петрович працює адвокатом, а не лікарем-травматологом у Львові.'  
Output:  
PERS: 'Микола Петрович'  
JOB: 'адвокатом'  
LOC: 'Львові'


Here is the text for analysis:  
Text: '{text}'

Please process the text and provide the output strictly in the specified format.
"""

In [ ]:
cot_prompt = """
**Thinking...**

1. I interpret the task as identifying all unique named entities in a given text. The entities belong to predefined categories, each with specific definitions (e.g., ART, DATE, DOC, etc.).

2. I will follow these steps:
   - Accept the provided input text.
   - Parse the text to identify potential entities.
   - Categorize each entity according to the given definitions.
   - Ensure uniqueness by filtering out duplicates.
   - Format the output strictly according to the specified template.

3. For clarity, I will use logical reasoning to ensure no entity is misclassified and to handle ambiguous cases transparently.

**Here is the text for analysis:**
Text: '{text}'

**Steps to Solve:**

1. **Parse the Input:**
   - Extract all potential named entities from the input text`.
   - Use linguistic patterns and context to determine entity boundaries.

2. **Classify Entities:**
   - For each entity identified in input text, match it to the most fitting category:
     - ART: Objects created by humans (e.g., tools, artworks).
     - DATE: Specific calendar dates.
     - DOC: Official or unofficial texts (e.g., contracts, reports).
     - JOB: Professional titles or occupations.
     - LOC: Geographic names.
     - MON: Monetary amounts including currency.
     - ORG: Organizations, including names and abbreviations.
     - PCT: Percentages.
     - PERIOD: Time intervals or durations.
     - PERS: Personal names.
     - QUANT: Measurable quantities (e.g., weight, distance, size).
     - TIME: Specific times of day.
     - MISC: Miscellaneous entities not fitting other categories.

3. **Ensure Uniqueness:**
   - Remove duplicates from the entities extracted from input text by comparing text and category.

4. **Review and Validate:**
   - Double-check classifications against definitions.
   - Confirm that the output includes all unique entities from input text.

**Thought...**

The input text `text` will be dynamically analyzed, and the results will be output in the required format.

This structured flow ensures the input text is central to the process and clearly defines where it is analyzed.
"""

In [ ]:
import re
import gc
import json
import torch
import pandas as pd

from transformers import AutoTokenizer
from vllm import LLM, SamplingParams

In [ ]:
import os
os.environ["TORCH_LOGS"] = "+dynamo"
os.environ["TORCHDYNAMO_VERBOSE"] = "1"

# Import torch dynamo and configure settings
import torch._dynamo
torch._dynamo.config.suppress_errors = True

In [ ]:
llm = LLM(model=MODEL_ID, device="cuda");
tokenizer = AutoTokenizer.from_pretrained(TOKENIZER_ID);

In [ ]:
from enum import Enum
from pydantic import BaseModel, constr
from vllm.sampling_params import GuidedDecodingParams


class LabelEnum(str, Enum):
    LOC = "LOC"
    ORG = "ORG"
    PERS = "PERS"
    MON = "MON"
    PCT = "PCT"
    DATE = "DATE"
    TIME = "TIME"
    PERIOD = "PERIOD"
    JOB = "JOB"
    DOC = "DOC"
    QUANT = "QUANT"
    ART = "ART"
    MISC = "MISC"


class Entity(BaseModel):
    label: LabelEnum
    text: constr(strip_whitespace=True)
    
    
class AllEntities(BaseModel):
    entities: list[Entity]
    
    def entities_to_json(self) -> str:
        return json.dumps([entity.model_dump() for entity in self.entities], indent=4, ensure_ascii=False)



json_schema = AllEntities.model_json_schema()
guided_decoding_params = GuidedDecodingParams(json=json_schema)
sampling_params = SamplingParams(guided_decoding=guided_decoding_params, max_tokens=512, temperature=0.1, repetition_penalty=1.2)

In [ ]:
def parse_ner_output(ner_output: str) -> str:
    pattern = r'"label":\s*"(.*?)",\s*"text":\s*"(.*?)"'
    matches = re.findall(pattern, ner_output)

    entities = [{"label": label, "text": text.strip()} for label, text in matches]

    unique_entities = {f"{entity['label']}:{entity['text']}": entity for entity in entities}.values()
    return json.dumps(list(unique_entities), indent=4, ensure_ascii=False)

In [ ]:
def process_entities_batch(texts, prompt):
    messages = [[{"role": "user", "content": prompt.format(text=row)}] for row in texts]
    text = tokenizer.apply_chat_template(messages, tokenize=False, add_generation_prompt=True, batch_size=12)

    outputs = [result.outputs[0].text for result in llm.generate(text, sampling_params)]

    results = []
    for output in outputs:
        try:
            results.append(AllEntities(**json.loads(output)).entities_to_json())
        except:
            results.append(parse_ner_output(output))

    torch.cuda.empty_cache()
    gc.collect()
    return results

## Execution
___

In [ ]:
df = pd.read_csv("test.csv")
texts = df["text"].to_list()

In [ ]:
!sudo apt-get -y update
!sudo apt-get -y install build-essential

In [ ]:
df["pred"] = process_entities_batch(texts, zero_shot_prompt)
df.to_csv(f"zero_shot.csv", index=False)

In [ ]:
df["pred"] = process_entities_batch(texts, few_shot_prompt)
df.to_csv(f"few_shot.csv", index=False)

In [ ]:
df["pred"] = process_entities_batch(texts, cot_prompt)
df.to_csv(f"CoT.csv", index=False)